In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
from pybedtools import BedTool
# import ipdb
import time
from tqdm import tqdm
import scipy.stats

# Processing ChIA-PET data

In [149]:
# Process ChIA-PET data
#  dataset ,point_name, value_name1,value_name2, value1, value2

def InInterval(s1,e1,e2,s3):
    return ((int(e1) >= np.array(e2)) * (np.array(e2)>=int(s1)))*((int(e1) >= np.array(s3)) * (np.array(s3)>=int(s1)))


dataset_name = 'LHG0052H.e500.clusters.cis.bothanchint_G100.PETcnt_G4.sorted.bedpe'
value_name1 = 'LoopNumber'
value_name2 = 'Tot_Loopcount'

Region_DF = pd.read_csv('Region_short_dm.bed',sep = '\t',names = ['chr','dm_start','dm_end','dmID',
                                                                  'A','B','C','D','E','F','G'])
Region_DF = Region_DF.iloc[:,0:4]
# Region_DF.head()


path = 'Minji_data/LHG0052H.e500.clusters.cis.bothanchint_G100.PETcnt_G4.sorted.bedpe'
PETloop_DF = pd.read_csv(path,sep = '\t',names = ['chrF','lp_L_start','lp_L_end','chrS','lp_R_start','lp_R_end','PET_count'])
PETloop_DF = PETloop_DF.iloc[:,[0,2,4,6]]

# PETloop_DF.head()
List = []
for i in range(len(Region_DF)):
# for i in range(1):
    Temploop_DF = PETloop_DF[PETloop_DF['chrF'] == Region_DF.loc[i,'chr']]
    InDmloop = Temploop_DF[InInterval(Region_DF.loc[i,'dm_start'],Region_DF.loc[i,'dm_end'],Temploop_DF['lp_L_end'],Temploop_DF['lp_R_start'])]
    loopNum = len(InDmloop)
    TotloopCount = np.sum(np.array(InDmloop['PET_count']))
    List.append([dataset_name,Region_DF.loc[i,'dmID'],value_name1,value_name2,loopNum,TotloopCount])

DF = pd.DataFrame(List,columns =['dataset','point_name','value_name1','value_name2',value_name1,value_name2])
DF.to_csv('Minji_data/Final_data_results/Reproduce_results/'+dataset_name+'_PETNumandCount'+'.csv',index=False)
# Max_iter = Region_short.count()
# for i in tqdm(range(Max_iter)):
#     NowRegion = BedTool(Region_short[i:i+1]).saveas()

# Processing ChIA-Drop data

In [115]:
# Extract ChIA-Drop data

dataset_name = 'SHG0180-181-195-196NR_hg38_RAD21_filt_comp_FDR_0.1_ALL'
value_name = 'ChIA-Drop: number of complexes (L+R+B)'
savepath = 'Minji_data/Final_data_results/Reproduce_results/'
path = 'Minji_data/Final_data_results/SHG_NR_RAD21/01ALL_dm/Tables/All_Results.csv'
DF = pd.read_csv(path)
DF.head()

DF_short = pd.DataFrame(DF['Total-None']).rename(columns = {'Total-None' : 'value'})
DF_short.insert(0,'value_name',[value_name]*len(DF))
DF_short.insert(0,'point_name',DF['LoopID'])
DF_short.insert(0,'dataset',[dataset_name]*len(DF))

DF_short.to_csv(savepath+dataset_name+'_'+value_name+'.csv',index=False)
# Mdata_short.insert(0,'dataset',[dataset_name]*len(Mdata))
# Mdata_short.insert(2,'value_name',[value_name]*len(Mdata))
# Mdata_short.head()

# Extracting intensity for each motif

In [100]:
# dataset ,point_name, value_name, value
# Motif nodes here

dataset_name = 'CTCF_motifs_STORM_hg38_Ext4kbBoth_with_supermotif_domain_id_v7.sorted.bed'
value_name = 'Cohesin_Drop_intensity'
Motifpath = 'Minji_data/'+dataset_name
savepath = 'Minji_data/Final_data_results/Reproduce_results/'


# Motifpath = 'Minji_data/CTCF_motifs_STORM_hg38_Ext4kbBoth_with_supermotif_for_casilio_loops.sorted.bed'
Mdata = pd.read_csv(Motifpath, sep = '\t',
                    names = ['chr','start','end','orientation','Mt_ID',
                             'dmID','Side',
                             'CTCF PET intensity','CTCF Drop intensity',
                             'Coh PET intensity','Coh Drop intensity'])

# Mdata = Mdata[Mdata['dmID'] != '.']
# Mdata.head()
Mdata_short = pd.DataFrame(Mdata['Mt_ID']).rename(columns = {'Mt_ID' : 'point_name'})
Mdata_short.insert(1,'value',Mdata['Coh Drop intensity'])
Mdata_short.insert(0,'dataset',[dataset_name]*len(Mdata))
Mdata_short.insert(2,'value_name',[value_name]*len(Mdata))
# Mdata_short.head()


Mdata_short.to_csv(savepath+dataset_name+'_'+value_name+'.csv',index=False)

# This is the plot function:  
Input data should have form: [dataset,point_name,value_name,value]
Or [dataset,point_name,value_name1,value_name2,value1,value2]

In [185]:
# Plot place

def mainplot(dataset,data1name,data2name,savepath,postfix,Multi = ''):
    # dataset: dataset that used for plotting
    # data1name: data for x-axis
    # data2name: data for y-axis
    # Multi: option for different valuei (i.e. Multi = 2 will plot value2, default is '')
    # savepath: path to save the plot
    # postfix: string that added to the end of the file name

#     dataset = 'CTCF_motifs_STORM_hg38_Ext4kbBoth_with_supermotif_domain_id_v7.sorted.bed_Cohesin'

#     data1name = 'CTCF_motifs_STORM_hg38_Ext4kbBoth_with_supermotif_domain_id_v7.sorted.bed_CTCF_Drop_intensity.csv'
    data1path = 'Minji_data/Final_data_results/Reproduce_results/'+data1name

#     data2name = 'CTCF_motifs_STORM_hg38_Ext4kbBoth_with_supermotif_domain_id_v7.sorted.bed_CTCF_PET_intensity.csv'
    data2path = 'Minji_data/Final_data_results/Reproduce_results/'+data2name

    DF1 = pd.read_csv(data1path)
    DF2 = pd.read_csv(data2path)

    # # Get top 20%
    # DF2 = DF2.sort_values(by=['value'], ascending=False)
    # DF2 = DF2.iloc[0:int(len(DF2)/5)]
    # # print(DF1.iloc[-1])

    if Multi == '':
        point_name_list = DF2['point_name']
        Data_list = []
        # Data_list = ['']*len(point_name_list)
        for i in range(len(point_name_list)):
        # for i in range(3):
            FindDF1 = DF1[DF1['point_name'] == DF2.iloc[i,1]]
        #     print(FindDF2)
#             if int(FindDF1.iloc[0,-1])> -1 and int(DF2.iloc[i,-1])> -1:
            Data_list.append([FindDF1.iloc[0,-1],DF2.iloc[i,-1]])
    #         print(DF1.iloc[i,1],DF1.iloc[i,-1],FindDF2.iloc[0,-1])
    # #         Data_list[i] = [0,0]
    #     else:
    #         Data_list[i] = [DF1.iloc[i,-1],FindDF2.iloc[0,-1]]
    else:
        point_name_list = DF2['point_name']
        Data_list = []
        # Data_list = ['']*len(point_name_list)
        for i in range(len(point_name_list)):
        # for i in range(3):
            FindDF1 = DF1[DF1['point_name'] == DF2.iloc[i,1]]
        #     print(FindDF2)
#             if int(FindDF1.iloc[0,-1])> -1 and int(DF2.iloc[i,-1])> -1:
            Data_list.append([FindDF1.iloc[0,-int(Multi)],DF2.iloc[i,-int(Multi)]])

    DataMatrix = np.array(Data_list)
    Pearson,_ = scipy.stats.pearsonr(DataMatrix[:,0],DataMatrix[:,1])
    Spearman,_ = scipy.stats.spearmanr(DataMatrix[:,0],DataMatrix[:,1])

    # fig = plt.figure()
    Max_x = np.max(DataMatrix[:,0])
    Min_x = np.min(DataMatrix[:,0])
    Mean_x = np.mean(DataMatrix[:,0])
    std_x = np.std(DataMatrix[:,0])
    std_y = np.std(DataMatrix[:,1])
    Mean_y = np.mean(DataMatrix[:,1])
    xx = [Min_x,Max_x]
    yy = Mean_y+std_y/std_x*Pearson*(xx-Mean_x)

    fig,ax = plt.subplots(1, 1, dpi = 150,figsize = (3, 3))
    ax.scatter(DataMatrix[:,0],DataMatrix[:,1], marker= '.', c = 'k',alpha=0.5 )
    ax.plot(xx,yy,c = 'b')
    if Multi == '':
        ax.set_xlabel(DF1.iloc[0,2],fontsize=8,rotation=0)
        ax.set_ylabel(DF2.iloc[0,2],fontsize=8,rotation=90)
    else:
        ax.set_xlabel(DF1.iloc[0,int(Multi)+1],fontsize=8,rotation=0)
        ax.set_ylabel(DF2.iloc[0,int(Multi)+1],fontsize=8,rotation=90)

    title = 'point:'+str(len(point_name_list))+' ,datasets: '+ dataset + ' ,Pearson:' + str(round(Pearson, 3)) + ' ,Spearman:' + str(round(Spearman, 3))
    ax.set_title(title, fontsize = 4)

#     savepath = 'Minji_data/Final_data_results/Reproduce_results/Plots/{}.png'
#     postfix = 'Cohesin_ChIA_PET_intvsCohesin_ChIA_PET_int_top20%Drop'
    plt.savefig(savepath.format(dataset+'_'+postfix),
                    dpi = 600, bbox_inches='tight')